In [16]:
import pandas as pd
import numpy as np

results_df = pd.read_csv("results/chronos_benchmark_results.csv")

In [17]:
results_df.head()

,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,exchange_rate/short,chronos_bolt_tiny,0.000010,0.000010,0.002109,1.253411,0.003749,0.003753,19.888833,0.003120,0.004798,0.003244,0.004178,Econ/Fin,1
1,exchange_rate/medium,chronos_bolt_tiny,0.000283,0.000283,0.011907,8.691109,0.025486,0.024767,119.859031,0.016835,0.025405,0.017968,0.014884,Econ/Fin,1
2,exchange_rate/long,chronos_bolt_tiny,0.000903,0.000903,0.019534,10.271974,0.032043,0.031064,166.443321,0.030048,0.044899,0.029188,0.024416,Econ/Fin,1
3,ercot/short,chronos_bolt_tiny,442683.000000,442683.000000,314.393826,0.977523,0.053302,0.055636,6.788732,665.344272,0.127664,0.060325,0.048371,Energy,1
4,ercot/medium,chronos_bolt_tiny,700258.933333,700258.933333,409.076432,1.310945,0.078551,0.074039,18.131886,836.814754,0.160880,0.078646,0.067144,Energy,1


In [18]:
dataset_names = results_df["dataset"].unique()

In [19]:
results_df = results_df[["dataset", "domain", "num_variates", "model",
    "eval_metrics/MASE[0.5]", "eval_metrics/mean_weighted_sum_quantile_loss"]]
results_df = results_df.rename(columns={"eval_metrics/MASE[0.5]": "MASE"})
results_df = results_df.rename(columns={"eval_metrics/mean_weighted_sum_quantile_loss": "Qloss"})

In [20]:
results_df["MASE"] = pd.to_numeric(results_df["MASE"], errors="coerce")
results_df["rank_MASE"] = results_df.groupby(["dataset"])["MASE"].rank(method="min")

results_df["Qloss"] = pd.to_numeric(results_df["Qloss"], errors="coerce")
results_df["rank_Qloss"] = results_df.groupby(["dataset"])["Qloss"].rank(method="min")

results_df["Rank"] = results_df["rank_MASE"] + results_df["rank_Qloss"] / 2

results_df = results_df[results_df["dataset"] != "dataset"]

results_df.head()

,dataset,domain,num_variates,model,MASE,Qloss,rank_MASE,rank_Qloss,Rank
0,exchange_rate/short,Econ/Fin,1,chronos_bolt_tiny,1.253411,0.004178,4.0,2.0,5.0
1,exchange_rate/medium,Econ/Fin,1,chronos_bolt_tiny,8.691109,0.014884,2.0,2.0,3.0
2,exchange_rate/long,Econ/Fin,1,chronos_bolt_tiny,10.271974,0.024416,3.0,3.0,4.5
3,ercot/short,Energy,1,chronos_bolt_tiny,0.977523,0.048371,4.0,4.0,6.0
4,ercot/medium,Energy,1,chronos_bolt_tiny,1.310945,0.067144,4.0,3.0,5.5


In [ ]:
def create_dataset_df(results_df, dataset_name):
    dataset_df = results_df[results_df["dataset"] == dataset_name]
    dataset_df = dataset_df.drop(columns=["dataset", "domain", "num_variates", "rank_MASE", "rank_Qloss"])
    dataset_df = dataset_df.rename(columns={"model": "Metric"})
    dataset_df = dataset_df.set_index("Metric")
    dataset_df = dataset_df.transpose()
    dataset_df = dataset_df.rename(columns={"chronos_bolt_base": "Chr.B.B",
                                            "chronos_bolt_small": "Chr.B.S",
                                            "chronos_bolt_mini": "Chr.B.M",
                                            "chronos-bolt-mini": "Chr.B.M",
                                            "chronos_bolt_tiny": "Chr.B.T",
                                            "timesfm1": "T.FM1",
                                            "timesfm2": "T.FM2",
                                            "morai_small": "Moi.S",
                                            "morai_base": "Moi.B",
                                            "morai_large": "Moi.L",
                                            })
    dataset_df["Best"] = dataset_df.idxmin(axis=1) # add columns best
    return dataset_df

In [ ]:
final_df = pd.DataFrame()

for name in dataset_names:
    dataset_df = create_dataset_df(results_df, name)
    dataset_df["dataset, term"] = name.replace("/", ", ")

    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df[["dataset, term", "index", "Chr.B.B", "Chr.B.S", "Chr.B.M",
        "Chr.B.T", "T.FM1", "T.FM2", "Moi.S", "Moi.B", "Moi.L", "Best"]]
    dataset_df = dataset_df.rename(columns={"index": "Metric"})
    dataset_df = dataset_df.rename_axis(index=" ")

    final_df = pd.concat([final_df, dataset_df], ignore_index=True)

In [23]:
final_df.head()

Metric,"dataset, term",Metric,Chr.B.B,Chr.B.S,Chr.B.M,Chr.B.T,Best
0,"exchange_rate, short",MASE,1.249064,0.802832,1.105789,1.253411,Chr.B.S
1,"exchange_rate, short",Qloss,0.004579,0.003843,0.004379,0.004178,Chr.B.S
2,"exchange_rate, short",Rank,5.000000,1.500000,3.500000,5.000000,Chr.B.S
3,"exchange_rate, medium",MASE,9.167733,9.054845,8.657556,8.691109,Chr.B.M
4,"exchange_rate, medium",Qloss,0.016081,0.014975,0.013126,0.014884,Chr.B.M


In [24]:
final_df.to_csv("results/chronos_benchmark.csv", index=False)

In [25]:
print(final_df.to_latex(index=False, float_format="%.3f"))

\begin{tabular}{llrrrrl}
\toprule
dataset, term & Metric & Chr.B.B & Chr.B.S & Chr.B.M & Chr.B.T & Best \\
\midrule
exchange_rate, short & MASE & 1.249 & 0.803 & 1.106 & 1.253 & Chr.B.S \\
exchange_rate, short & Qloss & 0.005 & 0.004 & 0.004 & 0.004 & Chr.B.S \\
exchange_rate, short & Rank & 5.000 & 1.500 & 3.500 & 5.000 & Chr.B.S \\
exchange_rate, medium & MASE & 9.168 & 9.055 & 8.658 & 8.691 & Chr.B.M \\
exchange_rate, medium & Qloss & 0.016 & 0.015 & 0.013 & 0.015 & Chr.B.M \\
exchange_rate, medium & Rank & 6.000 & 4.500 & 1.500 & 3.000 & Chr.B.M \\
exchange_rate, long & MASE & 11.062 & 9.700 & 9.506 & 10.272 & Chr.B.M \\
exchange_rate, long & Qloss & 0.029 & 0.024 & 0.022 & 0.024 & Chr.B.M \\
exchange_rate, long & Rank & 6.000 & 3.000 & 1.500 & 4.500 & Chr.B.M \\
ercot, short & MASE & 0.709 & 0.816 & 0.892 & 0.978 & Chr.B.B \\
ercot, short & Qloss & 0.033 & 0.041 & 0.044 & 0.048 & Chr.B.B \\
ercot, short & Rank & 1.500 & 3.000 & 4.500 & 6.000 & Chr.B.B \\
ercot, medium & MASE & 1.2

### More results

In [26]:
import json

with open("data/chronos_data_proprieties.json") as f:
    dataset_properties_map = json.load(f)

In [ ]:
final_df["Dataset"] = final_df["dataset, term"].apply(
    lambda x: x.split(", ")[0] if len(x.split(", ")) > 0 else "Unknown")
final_df["Domain"] = final_df["Dataset"].apply(lambda x: dataset_properties_map[x]["domain"]
    if x in dataset_properties_map else "Unknown")
final_df["Term"] = final_df["dataset, term"].apply(
    lambda x: x.split(", ")[1] if len(x.split(", ")) > 1 else "Unknown")
final_df["Frequency"] = final_df["Dataset"].apply(lambda x: dataset_properties_map[x]["freq"]
    if x in dataset_properties_map else "Unknown")

final_df = final_df[["Dataset", "Domain", "Term", "Frequency", "Metric",
    "Chr.B.B", "Chr.B.S", "Chr.B.M", "Chr.B.T", "T.FM1", "T.FM2",
    "Moi.S", "Moi.B", "Moi.L", "Best"]]

final_df.head()

Metric,Dataset,Domain,Term,Frequency,Metric,Chr.B.B,Chr.B.S,Chr.B.M,Chr.B.T,Best
0,exchange_rate,Econ/Fin,short,D,MASE,1.249064,0.802832,1.105789,1.253411,Chr.B.S
1,exchange_rate,Econ/Fin,short,D,Qloss,0.004579,0.003843,0.004379,0.004178,Chr.B.S
2,exchange_rate,Econ/Fin,short,D,Rank,5.000000,1.500000,3.500000,5.000000,Chr.B.S
3,exchange_rate,Econ/Fin,medium,D,MASE,9.167733,9.054845,8.657556,8.691109,Chr.B.M
4,exchange_rate,Econ/Fin,medium,D,Qloss,0.016081,0.014975,0.013126,0.014884,Chr.B.M
5,exchange_rate,Econ/Fin,medium,D,Rank,6.000000,4.500000,1.500000,3.000000,Chr.B.M
6,exchange_rate,Econ/Fin,long,D,MASE,11.061553,9.700047,9.506321,10.271974,Chr.B.M
7,exchange_rate,Econ/Fin,long,D,Qloss,0.028943,0.024171,0.021712,0.024416,Chr.B.M
8,exchange_rate,Econ/Fin,long,D,Rank,6.000000,3.000000,1.500000,4.500000,Chr.B.M
9,ercot,Energy,short,H,MASE,0.708769,0.816378,0.892499,0.977523,Chr.B.B


In [ ]:
domain_results = final_df.groupby(["Domain", "Metric"]).agg({
    "Chr.B.B": "mean",
    "Chr.B.S": "mean",
    "Chr.B.M": "mean",
    "Chr.B.T": "mean",
    "T.FM1": "mean",
    "T.FM2": "mean",
    "Moi.S": "mean",
    "Moi.B": "mean",
    "Moi.L": "mean",
}).reset_index()
domain_results["Best"] = domain_results[["Chr.B.B", "Chr.B.S", "Chr.B.M", "Chr.B.T", "T.FM1", "T.FM2",
    "Moi.S", "Moi.B", "Moi.L",]].idxmin(axis=1)

domain_results

In [ ]:
term_results = final_df.groupby(["Term", "Metric"]).agg({
    "Chr.B.B": "mean",
    "Chr.B.S": "mean",
    "Chr.B.M": "mean",
    "Chr.B.T": "mean",
    "T.FM1": "mean",
    "T.FM2": "mean",
    "Moi.S": "mean",
    "Moi.B": "mean",
    "Moi.L": "mean",
}).reset_index()
term_results["Best"] = term_results[["Chr.B.B", "Chr.B.S", "Chr.B.M", "Chr.B.T", "T.FM1", "T.FM2",
    "Moi.S", "Moi.B", "Moi.L",]].idxmin(axis=1)

term_results

In [ ]:
frequency_results = final_df.groupby(["Frequency", "Metric"]).agg({
    "Chr.B.B": "mean",
    "Chr.B.S": "mean",
    "Chr.B.M": "mean",
    "Chr.B.T": "mean",
    "T.FM1": "mean",
    "T.FM2": "mean",
    "Moi.S": "mean",
    "Moi.B": "mean",
    "Moi.L": "mean",
}).reset_index()
frequency_results["Frequency"] = frequency_results["Frequency"].map({
    "H": "1 hour",
    "D": "1 day",
    "W": "1 week",
})
# sort by frequency
frequency_results = frequency_results.sort_values(by=["Frequency", "Metric"])

frequency_results["Best"] = frequency_results[["Chr.B.B", "Chr.B.S", "Chr.B.M", "Chr.B.T", "T.FM1", "T.FM2",
    "Moi.S", "Moi.B", "Moi.L"]].idxmin(axis=1)

frequency_results # same as domain

In [ ]:
print(domain_results.to_latex(index=False, float_format="%.3f"))

In [ ]:
print(term_results.to_latex(index=False, float_format="%.3f"))